In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Project-2/Sport-Salaries/')


In [36]:
nba_players_file = "nba_players.csv"
nba_players_df = pd.read_csv(nba_players_file)
nba_players_df.head()


,_id,birthDate,birthPlace,career_AST,career_FG%,career_FG3%,career_FT%,career_G,career_PER,career_PTS,...,draft_pick,draft_round,draft_team,draft_year,height,highSchool,name,position,shoots,weight
0,abdelal01,"June 24, 1968","Cairo, Egypt",0.3,50.2,0.0,70.1,256,13.0,5.7,...,25th overall,1st round,Portland Trail Blazers,1990,6-10,"Bloomfield in Bloomfield, New Jersey",Alaa Abdelnaby,Power Forward,Right,240lb
1,abdulza01,"April 7, 1946","Brooklyn, New York",1.2,42.8,NaN,72.8,505,15.1,9.0,...,5th overall,1st round,Cincinnati Royals,1968,6-9,"John Jay in Brooklyn, New York",Zaid Abdul-Aziz,Power Forward and Center,Right,235lb
2,abdulka01,"April 16, 1947","New York, New York",3.6,55.9,5.6,72.1,1560,24.6,24.6,...,1st overall,1st round,Milwaukee Bucks,1969,7-2,"Power Memorial in New York, New York",Kareem Abdul-Jabbar,Center,Right,225lb
3,abdulma02,"March 9, 1969","Gulfport, Mississippi",3.5,44.2,35.4,90.5,586,15.4,14.6,...,3rd overall,1st round,Denver Nuggets,1990,6-1,"Gulfport in Gulfport, Mississippi",Mahmoud Abdul-Rauf,Point Guard,Right,162lb
4,abdulta01,"November 3, 1974","Maisons Alfort, France",1.1,41.7,23.7,70.3,236,11.4,7.8,...,11th overall,1st round,Sacramento Kings,1997,6-6,"Lycee Aristide Briand in Evreux, France",Tariq Abdul-Wahad,Shooting Guard,Right,223lb


In [37]:
# Create a filtered dataframe from specific columns
player_cols = ["_id", "name", "position"]
players_transformed = nba_players_df[player_cols].copy()

# Rename the column headers
players_transformed = players_transformed.rename(columns={"_id": "id",
                                                          "name": "player_name"})

# Clean the data by dropping duplicates and setting the index
players_transformed.drop_duplicates("id", inplace=True)
players_transformed.set_index("id", inplace=True)

players_transformed.head()


,player_name,position
id,,
abdelal01,Alaa Abdelnaby,Power Forward
abdulza01,Zaid Abdul-Aziz,Power Forward and Center
abdulka01,Kareem Abdul-Jabbar,Center
abdulma02,Mahmoud Abdul-Rauf,Point Guard
abdulta01,Tariq Abdul-Wahad,Shooting Guard


In [38]:
nba_salaries_file = "nba_salaries.csv"
nba_salaries_df = pd.read_csv(nba_salaries_file)
nba_salaries_df.head()


,league,player_id,salary,season,season_end,season_start,team
0,NBA,abdelal01,395000,1990-91,1991,1990,Portland Trail Blazers
1,NBA,abdelal01,494000,1991-92,1992,1991,Portland Trail Blazers
2,NBA,abdelal01,500000,1992-93,1993,1992,Boston Celtics
3,NBA,abdelal01,805000,1993-94,1994,1993,Boston Celtics
4,NBA,abdelal01,650000,1994-95,1995,1994,Sacramento Kings


In [39]:
# Create a filtered dataframe from specific columns
salaries_cols = ["player_id", "season_end", "salary",  "team"]
salaries_transformed = nba_salaries_df[salaries_cols].copy()

# Rename the column headers
salaries_transformed = salaries_transformed.rename(columns={"player_id": "id",
                                                            "season_end" : "year",
                                                            "salary" : "salary",
                                                           })

# Clean the data by dropping duplicates and setting the index
salaries_transformed.set_index("id", inplace=True)

salaries_transformed.head()


,year,salary,team
id,,,
abdelal01,1991,395000,Portland Trail Blazers
abdelal01,1992,494000,Portland Trail Blazers
abdelal01,1993,500000,Boston Celtics
abdelal01,1994,805000,Boston Celtics
abdelal01,1995,650000,Sacramento Kings


In [40]:
# Merge two dataframes using an outer join
merged_info = pd.merge(players_transformed, salaries_transformed, on="id", how="outer")
merged_info = merged_info.dropna()
merged_info.head()


,player_name,position,year,salary,team
id,,,,,
abdelal01,Alaa Abdelnaby,Power Forward,1991.0,395000.0,Portland Trail Blazers
abdelal01,Alaa Abdelnaby,Power Forward,1992.0,494000.0,Portland Trail Blazers
abdelal01,Alaa Abdelnaby,Power Forward,1993.0,500000.0,Boston Celtics
abdelal01,Alaa Abdelnaby,Power Forward,1994.0,805000.0,Boston Celtics
abdelal01,Alaa Abdelnaby,Power Forward,1995.0,650000.0,Sacramento Kings


In [43]:
merged_cols = ["year", "player_name", "position", "salary", "team"]
nba_transformed = merged_info[merged_cols].copy()

nba_transformed = nba_transformed[["year", "player_name", "position", "salary", "team"]]

# Create a filtered dataframe from specific columns
nba_cols = ["player_name", "position", "team", "salary", "year"]
nba_transformed = nba_transformed[nba_cols].copy()

# Rename the column headers
nba_transformed = nba_transformed.rename(columns={"player_name": "player_name",
                                                          "position": "position",
                                                          "team": "team",
                                                          "salary": "avg_salary",
                                                          "year": "year"})

# Clean the data by dropping NA's, clean avg_salary, and setting the index
nba_transformed = nba_transformed.dropna(axis = 0, how ='any') 
nba_transformed = nba_transformed.sort_values('team', ascending=False).drop_duplicates(subset=['player_name', 'position', 'year'])

#nba_transformed_cond = nba_transformed['avg_salary'] != "$NaN"
#nba_transformed = nba_transformed[mlb_transformed_cond]
nba_transformed.astype({'year': 'int32'}).dtypes
pd.set_option('precision', 0)
nba_transformed.set_index("year", inplace=True)
nba_transformed.sort_values(by=['year'])
nba_transformed.head()


,player_name,position,team,avg_salary
year,,,,
2002,Hubert Davis,Point Guard and Shooting Guard,Washington Wizards,2e+06
2007,Jarvis Hayes,Shooting Guard and Small Forward,Washington Wizards,3e+06
2002,Rod Strickland,Shooting Guard and Point Guard,Washington Wizards,4e+06
2001,Rod Strickland,Shooting Guard and Point Guard,Washington Wizards,1e+07
2000,Rod Strickland,Shooting Guard and Point Guard,Washington Wizards,1e+07


In [44]:
#connect to postgres
connection_string = "postgres:akopova@localhost:5433/sports_db"
engine = create_engine(f'postgresql://{connection_string}')


In [49]:
pd.read_sql_query('CREATE TABLE nba_player_salary(year INT, player_name VARCHAR(500), position VARCHAR(100), team VARCHAR(100), avg_salary INT, PRIMARY KEY (year, player_name, position))', con=engine).head()


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [46]:
engine.table_names()


['nfl_player_salary', 'mlb_player_salary', 'nba_player_salary']

In [50]:
nba_transformed.to_sql(name='nba_player_salary', con=engine, if_exists='append', index=True)


In [51]:
pd.read_sql_query('select * from nba_player_salary', con=engine).head()


,year,player_name,position,team,avg_salary
0,2002,Hubert Davis,Point Guard and Shooting Guard,Washington Wizards,2418750
1,2007,Jarvis Hayes,Shooting Guard and Small Forward,Washington Wizards,2564127
2,2002,Rod Strickland,Shooting Guard and Point Guard,Washington Wizards,3750000
3,2001,Rod Strickland,Shooting Guard and Point Guard,Washington Wizards,10000000
4,2000,Rod Strickland,Shooting Guard and Point Guard,Washington Wizards,10000000
